In [1]:
!pip install pytesseract mss pillow pynput

In [3]:
# antes se debe instalar tesseract : brew install tesseract-ocr libtesseract-dev tesseract, en Mac o en linux con apt

import pytesseract
from PIL import Image

def ocr_image(image_path):
    try:
        # Cargar la imagen usando PIL (Python Imaging Library)
        image = Image.open(image_path)
        
        # Utilizar pytesseract para realizar OCR en la imagen
        text = pytesseract.image_to_string(image, lang='eng')  # lang='eng' para inglés
        
        return text.strip()
    
    except Exception as e:
        print("Error al procesar la imagen:", e)
        return None

if __name__ == "__main__":
    # Ruta de la imagen que deseas procesar
    image_path = "./image_maquina.jpg" # Cambia esto por la ruta de tu imagen

    # Realizar OCR en la imagen y obtener el texto resultante
    extracted_text = ocr_image(image_path)

    if extracted_text:
        print("Texto extraído:")
        print(extracted_text)
    else:
        print("No se pudo extraer texto de la imagen.")


Error al procesar la imagen: cannot identify image file './AVISO ESPECIAL Nro. 01.pdf'
No se pudo extraer texto de la imagen.


In [22]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image
import pytesseract

def ocr_image(image_path):
    try:
        # Cargar la imagen usando PIL (Python Imaging Library)
        image = Image.open(image_path)
        
        # Utilizar pytesseract para realizar OCR en la imagen
        text = pytesseract.image_to_string(image, lang='eng')  # lang='eng' para inglés
        
        return text.strip()
    
    except Exception as e:
        print("Error al procesar la imagen:", e)
        return None

def browse_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg")])
    if file_path:
        image_entry.delete(0, tk.END)
        image_entry.insert(0, file_path)
        process_image()

def process_image():
    image_path = image_entry.get()
    if image_path:
        extracted_text = ocr_image(image_path)
        if extracted_text:
            text_box.delete(1.0, tk.END)
            text_box.insert(tk.END, extracted_text)
        else:
            text_box.delete(1.0, tk.END)
            text_box.insert(tk.END, "No se pudo extraer texto de la imagen.")

# Crear la ventana principal
root = tk.Tk()
root.title("OCR en Imágenes")
root.geometry("500x400")

# Etiqueta y entrada para mostrar la ruta de la imagen seleccionada
image_label = tk.Label(root, text="Imagen:")
image_label.pack(pady=5)

image_entry = tk.Entry(root, width=40)
image_entry.pack(pady=5)

# Botón para agregar una imagen
browse_button = tk.Button(root, text="Agregar Imagen", command=browse_image)
browse_button.pack(pady=10)

# Cuadro de texto para visualizar el texto convertido
text_box = tk.Text(root, wrap=tk.WORD, width=60, height=20)
text_box.pack(pady=5)

root.mainloop()


2023-07-25 16:12:20.903 python[61501:761765] png;*.jpg is not a valid allowedFileType because it cannot be converted to a UTType
2023-07-25 16:12:24.221 python[61501:761765] png;*.jpg is not a valid allowedFileType because it cannot be converted to a UTType
2023-07-25 16:12:25.327 python[61501:761765] png;*.jpg is not a valid allowedFileType because it cannot be converted to a UTType


In [29]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image
import pytesseract
import mss
import mss.tools

def ocr_image(image_path):
    try:
        # Cargar la imagen usando PIL (Python Imaging Library)
        image = Image.open(image_path)
        
        # Utilizar pytesseract para realizar OCR en la imagen
        text = pytesseract.image_to_string(image, lang='eng')  # lang='eng' para inglés
        
        return text.strip()
    
    except Exception as e:
        print("Error al procesar la imagen:", e)
        return None

def capture_area():
    with mss.mss() as sct:
        # Capturar el área seleccionada o todo el escritorio
        screenshot = sct.shot(mon=-1)

    if screenshot:
        image_entry.delete(0, tk.END)
        image_entry.insert(0, screenshot)
        process_image()

def process_image():
    image_path = image_entry.get()
    if image_path:
        extracted_text = ocr_image(image_path)
        if extracted_text:
            text_box.delete(1.0, tk.END)
            text_box.insert(tk.END, extracted_text)
        else:
            text_box.delete(1.0, tk.END)
            text_box.insert(tk.END, "No se pudo extraer texto de la imagen.")

# Crear la ventana principal
root = tk.Tk()
root.title("OCR - Captura de Área")
root.geometry("500x400")

# Etiqueta y entrada para mostrar la ruta de la imagen seleccionada
image_label = tk.Label(root, text="Imagen:")
image_label.pack(pady=5)

image_entry = tk.Entry(root, width=40)
image_entry.pack(pady=5)

# Botón para capturar un área del escritorio
capture_button = tk.Button(root, text="Capturar", command=capture_area)
capture_button.pack(pady=10)

# Cuadro de texto para visualizar el texto convertido
text_box = tk.Text(root, wrap=tk.WORD, width=60, height=20)
text_box.pack(pady=5)

root.mainloop()


In [2]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import mss

def capture_screen():
    with mss.mss() as sct:
        # Capturar toda la pantalla
        screenshot = sct.shot(output='screen.png')

    if screenshot:
        # Mostrar la captura completa de la pantalla
        image = Image.open("screen.png")
        image.thumbnail((800, 600))  # Reducir el tamaño de la imagen para mostrarla en la ventana
        imgtk = ImageTk.PhotoImage(image)

        image_label.config(image=imgtk)
        image_label.image = imgtk  # Mantener una referencia para evitar que la imagen se borre de la memoria

def select_area(event):
    global x1, y1, x2, y2
    x1, y1 = event.x, event.y

def release_area(event):
    global x1, y1, x2, y2
    x2, y2 = event.x, event.y

    # Recortar el área seleccionada y guardarla en un archivo
    screenshot = Image.open("screen.png")
    area = (min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2))
    cropped_image = screenshot.crop(area)

    file_path = filedialog.asksaveasfilename(defaultextension='.png', filetypes=[("PNG", "*.png")])
    if file_path:
        cropped_image.save(file_path)

# Crear la ventana principal
root = tk.Tk()
root.title("Capturar Área de Pantalla")
root.geometry("800x600")

# Etiqueta para mostrar la captura de pantalla
image_label = tk.Label(root)
image_label.pack()

# Botón para capturar toda la pantalla
capture_button = tk.Button(root, text="Capturar", command=capture_screen)
capture_button.pack(pady=10)

# Bindings para seleccionar un área con un click, arrastrar y soltar
x1, y1, x2, y2 = 0, 0, 0, 0
image_label.bind("<ButtonPress-1>", select_area)
image_label.bind("<B1-Motion>", select_area)
image_label.bind("<ButtonRelease-1>", release_area)

root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/miguelandrade/opt/anaconda3/lib/python3.9/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/var/folders/ty/3y6dblhx0h1d076pxyk3sh700000gn/T/ipykernel_68448/234205313.py", line 35, in release_area
    cropped_image.save(file_path)
  File "/Users/miguelandrade/opt/anaconda3/lib/python3.9/site-packages/PIL/Image.py", line 2320, in save
    save_handler(self, fp, filename)
  File "/Users/miguelandrade/opt/anaconda3/lib/python3.9/site-packages/PIL/PngImagePlugin.py", line 1374, in _save
    ImageFile._save(im, _idat(fp, chunk), [("zip", (0, 0) + im.size, 0, rawmode)])
  File "/Users/miguelandrade/opt/anaconda3/lib/python3.9/site-packages/PIL/ImageFile.py", line 510, in _save
    encoder.setimage(im.im, b)
SystemError: tile cannot extend outside image


In [14]:
!pip install pyscreenshot

In [27]:
import pyscreenshot
import time
# im = pyscreenshot.grab()
im = pyscreenshot.grab(bbox=(50, 50, 450, 350))

dateNow = time.localtime()
print(dateNow.tm_hour,dateNow.tm_min,dateNow.tm_sec)

im.save("screenshot"+str(dateNow.tm_hour)+"-"+str(dateNow.tm_min)+"-"+str(dateNow.tm_sec)+".png")

16 47 21


In [ ]:
!

In [42]:
import pyscreenshot
import time
from pynput.mouse import Listener

start_x, start_y, end_x, end_y = 0, 0, 0, 0

def on_click(x, y, button, pressed):
    global start_x, start_y, end_x, end_y
    if pressed:
        start_x, start_y = x, y
    else:
        end_x, end_y = x, y
        return False

def get_bounding_box_coordinates():
    with Listener(on_click=on_click) as listener:
        listener.join()

dateNow = time.localtime()

# Capturar el bbox con el ratón
get_bounding_box_coordinates()
im = pyscreenshot.grab(bbox=(start_x, start_y, end_x, end_y))

# Guardar la captura de pantalla con el bbox
im.save(f"screenshot_{dateNow.tm_hour}-{dateNow.tm_min}-{dateNow.tm_sec}.png")


This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.


In [46]:
import pyscreenshot
import time
from pynput.mouse import Listener, Controller

start_x, start_y, end_x, end_y = 0, 0, 0, 0

def on_click(x, y, button, pressed):
    global start_x, start_y, end_x, end_y
    if pressed:
        start_x, start_y = x, y
    else:
        end_x, end_y = x, y
        return False

def get_bounding_box_coordinates():
    with Listener(on_click=on_click) as listener:
        listener.join()

dateNow = time.localtime()

# Deshabilitar el ratón temporalmente
mouse = Controller()
original_pos = mouse.position  # Guardar la posición original del ratón
mouse.position = (-1, -1)  # Mover el ratón a una posición fuera de la pantalla para deshabilitar eventos de movimiento

# Capturar el bbox con el ratón
get_bounding_box_coordinates()
im = pyscreenshot.grab(bbox=(start_x, start_y, end_x, end_y))

# Guardar la captura de pantalla con el bbox
im.save(f"screenshot_{dateNow.tm_hour}-{dateNow.tm_min}-{dateNow.tm_sec}.png")

# Volver a habilitar el ratón
mouse.position = original_pos  # Restaurar la posición original del ratón


This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.


In [7]:
import pyscreenshot
import time
from pynput.mouse import Listener, Controller

start_x, start_y, end_x, end_y = 0, 0, 0, 0

def on_click(x, y, button, pressed):
    global start_x, start_y, end_x, end_y
    if pressed:
        start_x, start_y = x, y
    else:
        end_x, end_y = x, y
        return False

def get_bounding_box_coordinates():
    with Listener(on_click=on_click) as listener:
        listener.join()

dateNow = time.localtime()

# Capturar el bbox con el ratón
get_bounding_box_coordinates()
im = pyscreenshot.grab(bbox=(start_x, start_y, end_x, end_y))

# Guardar la captura de pantalla con el bbox
im.save(f"screenshot_{dateNow.tm_hour}-{dateNow.tm_min}-{dateNow.tm_sec}.png")


This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.


In [8]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import pyscreenshot
import time

start_x, start_y, end_x, end_y = 0, 0, 0, 0
cropped_img = None
image_refs = []  # Lista para mantener las referencias a las imágenes

def on_click(event):
    global start_x, start_y
    start_x, start_y = event.x, event.y

def on_release(event):
    global end_x, end_y
    end_x, end_y = event.x, event.y

    # Cerrar la ventana de selección
    root.quit()

# Crear la ventana de selección
root = tk.Tk()
root.title("Selección de Área")
root.geometry("800x600")

# Tomar captura de pantalla para mostrarla en la ventana
screenshot = pyscreenshot.grab()
img = ImageTk.PhotoImage(screenshot)
image_refs.append(img)  # Agregar la referencia a la lista
screenshot_label = tk.Label(root, image=img)
screenshot_label.pack()

# Bindings para capturar las coordenadas del área seleccionada
screenshot_label.bind("<ButtonPress-1>", on_click)
screenshot_label.bind("<B1-Motion>", on_release)

# Mostrar la ventana emergente
root.mainloop()

# Asegurarnos de que las coordenadas estén en el orden correcto
if end_x < start_x:
    start_x, end_x = end_x, start_x
if end_y < start_y:
    start_y, end_y = end_y, start_y

# Capturar el bbox con el ratón
im = pyscreenshot.grab(bbox=(start_x, start_y, end_x, end_y))

# Crear una ventana emergente para que el usuario pueda recortar la imagen seleccionada
root = tk.Tk()
root.title("Recortar Imagen")

# Mostrar la imagen seleccionada en la nueva ventana
cropped_img = ImageTk.PhotoImage(im)
image_refs.append(cropped_img)  # Agregar la referencia a la lista
cropped_label = tk.Label(root, image=cropped_img)
cropped_label.pack()

# Función para guardar la porción recortada de la imagen
def save_cropped_image():
    file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG", "*.png"), ("JPEG", "*.jpg")])
    if file_path:
        im.save(file_path)

# Botón para guardar la porción recortada de la imagen
save_button = tk.Button(root, text="Guardar", command=save_cropped_image)
save_button.pack()

root.mainloop()


TclError: image "pyimage6" doesn't exist

In [1]:
import sys
import numpy as np
from PyQt5.QtWidgets import QApplication, QMainWindow, QGraphicsView, QGraphicsScene, QGraphicsPixmapItem, QPushButton, QVBoxLayout, QWidget
from PyQt5.QtGui import QImage, QPixmap
import pyscreenshot

class ScreenshotWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Selección de Área")
        self.setGeometry(100, 100, 800, 600)

        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)

        self.layout = QVBoxLayout(self.central_widget)
        self.view = QGraphicsView(self)
        self.scene = QGraphicsScene(self)
        self.view.setScene(self.scene)
        self.layout.addWidget(self.view)

        self.capture_button = QPushButton("Capturar y Guardar", self)
        self.capture_button.clicked.connect(self.capture_and_save)
        self.layout.addWidget(self.capture_button)

        self.start_x, self.start_y, self.end_x, self.end_y = 0, 0, 0, 0

        self.update_screenshot()

    def update_screenshot(self):
        screenshot = pyscreenshot.grab()
        screenshot_np = np.array(screenshot)  # Convertir a arreglo numpy
        height, width, channel = screenshot_np.shape
        screenshot_qimage = QImage(screenshot_np, width, height, width * channel, QImage.Format_RGB888)
        self.screenshot_pixmap = QGraphicsPixmapItem()
        self.screenshot_pixmap.setPixmap(QPixmap.fromImage(screenshot_qimage))
        self.scene.clear()
        self.scene.addItem(self.screenshot_pixmap)
        self.view.setScene(self.scene)

    def mousePressEvent(self, event):
        self.start_x, self.start_y = event.x(), event.y()

    def mouseReleaseEvent(self, event):
        self.end_x, self.end_y = event.x(), event.y()
        self.capture_button.setEnabled(True)

    def capture_and_save(self):
        if self.start_x == self.end_x or self.start_y == self.end_y:
            return

        x1, x2 = sorted([self.start_x, self.end_x])
        y1, y2 = sorted([self.start_y, self.end_y])
        bbox = (x1, y1, x2, y2)

        screenshot = pyscreenshot.grab(bbox=bbox)
        screenshot.save("captured_area.png")

        self.capture_button.setEnabled(False)
        self.update_screenshot()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = ScreenshotWindow()
    window.show()
    sys.exit(app.exec_())


SystemExit: 0

/Users/miguelandrade/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [11]:
!pip install PyQt5 pyqtgraph pyscreenshot pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 14.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 17.7 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.


In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QGraphicsView, QGraphicsScene, QGraphicsRectItem, QVBoxLayout, QWidget, QPushButton
import pyqtgraph as pg
import pyscreenshot
from datetime import datetime

class ScreenshotWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Captura de Pantalla")
        self.setGeometry(100, 100, 800, 600)

        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)

        self.layout = QVBoxLayout(self.central_widget)
        self.view = QGraphicsView(self)
        self.scene = QGraphicsScene(self)
        self.view.setScene(self.scene)
        self.layout.addWidget(self.view)

        self.capture_button = QPushButton("Capturar", self)
        self.capture_button.clicked.connect(self.capture_screenshot)
        self.layout.addWidget(self.capture_button)

        self.roi = None

    def capture_screenshot(self):
        screenshot = pyscreenshot.grab()
        screenshot_np = screenshot.convert("RGB")
        screenshot_qimage = pg.ImageItem(screenshot_np)
        self.scene.clear()
        self.scene.addItem(screenshot_qimage)

        # Crear el ROI
        self.roi = pg.RectROI([100, 100], [200, 150], pen=(0, 9), scaleSnap=True, translateSnap=True)
        screenshot_qimage.setZValue(10)
        self.scene.addItem(self.roi)

        self.capture_button.setEnabled(False)
        self.roi.sigRegionChangeFinished.connect(self.on_roi_finished)

    def on_roi_finished(self):
        if self.roi is not None:
            x1, y1 = self.roi.pos()
            x2, y2 = self.roi.pos() + self.roi.size()
            bbox = (x1, y1, x2, y2)

            screenshot = pyscreenshot.grab(bbox=bbox)

            date_time_str = datetime.now().strftime("%Y%m%d_%H%M%S")
            file_path = f"captured_area_{date_time_str}.png"
            screenshot.save(file_path)

            self.roi.sigRegionChangeFinished.disconnect(self.on_roi_finished)
            self.scene.removeItem(self.roi)
            self.roi = None

            self.capture_button.setEnabled(True)
            self.capture_screenshot()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = ScreenshotWindow()
    window.show()
    sys.exit(app.exec_())


AttributeError: view

AttributeError: view

SystemExit: 0

/Users/miguelandrade/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


pdf

In [4]:
pip install PyMuPDF


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 7.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import fitz  # PyMuPDF
import pytesseract
from PIL import Image

def ocr_pdf(pdf_path):
    try:
        # Cargar el PDF usando PyMuPDF
        pdf_document = fitz.open(pdf_path)
        
        text = ""
        for page_num in range(pdf_document.page_count):
            page = pdf_document[page_num]
            
            # Convertir la página en una imagen PIL
            image = page.get_pixmap()
            image_pil = Image.frombytes("RGB", [image.width, image.height], image.samples)
            
            # Utilizar pytesseract para realizar OCR en la imagen
            page_text = pytesseract.image_to_string(image_pil, lang='eng')
            text += page_text + "\n"
        
        pdf_document.close()
        
        return text.strip()
    
    except Exception as e:
        print("Error al procesar el PDF:", e)
        return None

if __name__ == "__main__":
    # Directorio que contiene los archivos PDF a procesar
    pdf_directory = "./"  # Cambia esto por la ruta de tu directorio
    
    # Obtener la lista de archivos PDF en el directorio
    pdf_files = [file for file in os.listdir(pdf_directory) if file.endswith(".pdf")]
    
    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_directory, pdf_file)
        
        # Realizar OCR en el PDF y obtener el texto resultante
        extracted_text = ocr_pdf(pdf_path)
        
        if extracted_text:
            print("Texto extraído de", pdf_file, ":")
            print(extracted_text)
            print("-" * 50)
        else:
            print("No se pudo extraer texto de", pdf_file)
            print("-" * 50)


Texto extraído de CV MIGUEL 2023_python.pdf :
Curriculum Vitae

Miguel Octavio Andrade Pereira

Direcciin ‘av. Joséleal 1230 Lince
nt 70671271

Lugar nacimiento: Lima

Fecha de Nacimiento: 16/02/1992

Teléfono Celular: 930609470

Correo electrénico:

miguel andrade@unmem.edu pe

Formacién académica

Formacién Universitaria

Universidad Nacional de San
Marcos

Facultad de Ciencias Flsicas - Escuela
-Académico Profesional de Fisica

‘Area C: Medio Ambiente y Ciencias Espacales

Mayor

Grado: Gachiller
‘Ajo de Ingreso: 2012
‘Afio de Egreso: 2017

Universidad Nacional Mayor de San
Marcos

IMaestia en Ciencias Fiscas ~ Mencién en
Geofisica

Condicidn:Egresado

2018-2020

Instituto Superior Tecnolégico - sé Pardo
Computaciéne Informatica
‘io de Ingreso: 2020,

‘Actualidad: Egresado

Instituto Cultural Peruano Norte Americano
Ingles Avanzado 4

Nivel de Speaking: Ato

Nivel de Listening Alto

Nivel de Writing: Alto

CCertificad a Nivel Intermedio Completo
Excuela Latinoamericana de Gaia

2022

In [8]:
!pip install opengpt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.2/798.2 kB 9.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 11.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.1/368.1 kB 7.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 8.7 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 4.6 MB/s eta 0:00:0000:0100:01m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━

In [7]:
import opengpt

# Crear un modelo OpenGPT
model = opengpt.GPT()

# Traducir una oración del inglés al español
translation = model.translate("Hello world!", "en", "es")

# Imprimir la traducción
print(translation)

ModuleNotFoundError: No module named 'opengpt'